In [32]:
import time
import matplotlib.pyplot as plt
import numpy as np 
%matplotlib inline
import flowdyn.mesh           as mesh
import flowdyn.xnum           as xnum
import flowdyn.integration    as tnum
import flowdyn.modelphy.euler as euler
import flowdyn.modeldisc      as modeldisc
#
plt.rcParams['figure.dpi'] = 120
plt.rcParams['savefig.dpi'] = 120
plt.rcParams["animation.html"] = "jshtml"  # for matplotlib 2.1 and above, uses JavaScript

In [58]:
nx = 200
meshsim  = mesh.unimesh(ncell=nx,  length=10.)

def S(x):
    return 1.-.5*np.exp(-.5*(x-5.)**2)

plt.plot(meshsim.centers(), S(meshsim.centers())) ; plt.ylim(0,1)

model = euler.nozzle(sectionlaw=S)
bcL = { 'type': 'insub',  'ptot': 1.1, 'rttot': 1. }
bcR = { 'type': 'outsub', 'p': 1. }

rhs    = modeldisc.fvm(model, meshsim, xnum.muscl(xnum.vanalbada), bcL=bcL, bcR=bcR)
solver = tnum.rk3ssp(meshsim, rhs)

# computation
#
nsol    = 100  # number of intermediate resultats (for animation)
endtime = 100. # final physical time
cfl     = 1.

finit = rhs.fdata_fromprim([  1., 0.1, 1. ]) # rho, u, p
#finit = fsol[-1] ; finit.set_time(0)

fsol = solver.solve(finit, cfl, np.linspace(0., endtime, nsol+1))
solver.show_perf()

In [59]:
# Figure / Plot of final results
varname=['pressure', 'mach']
line = varname[:]
fig, ax = plt.subplots(1, 2, figsize=(10,4))
for i, var in zip(range(len(varname)),varname):
    ax[i].set_ylabel(var) ; ax[i].set_ylim(0., 1.1*np.max(fsol[-1].phydata(var)))
    ax[i].grid(linestyle='--', color='0.5')
    line[i], = fsol[-1].plot(var, 'k-', axes=ax[i])

In [60]:
import matplotlib.animation as anim
#
def animate(k):
    #i = min(k, nsol)
    for i, var in zip(range(len(varname)),varname):
        fsol[k].set_plotdata(line[i], var)
    return line

ani = anim.FuncAnimation(fig=fig, func=animate, frames=range(nsol+1), interval=100)#, blit=True)
ani